In [44]:
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd
import timeit
start = timeit.default_timer()

In [45]:
model = pybamm.lithium_ion.DFN()
param = pybamm.ParameterValues("OKane2023")

In [46]:
# C/10 discharge/charge run at start and after every 78 ageing cycles 
    # define experiment
V_max = 4.2;        
V_min = 2.5; 
charge_time_mins = 60 * 4.86491/5
""" exp_AGE_text = [(
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C for {charge_time_mins} minutes",
    ),  ] """
exp_AGE_text = [(
    f"Discharge at 1 C for 2 minutes", 
    f"Charge at 0.3 C for 2 minutes",
    ),  ]
# step index for ageing
step_AGE_CD =0;   step_AGE_CC =1;   step_AGE_CV =2;

exp_RPT_text = [ (
    f"Hold at {V_max}V until C/100",
    f"Discharge at 0.1C until {V_min} V",  
    "Rest for 1 hours",  
    f"Charge at 0.1C until {V_max} V",
    "Rest for 1 hours",
    f"Discharge at 1C until {V_min} V",  
    f"Hold at {V_min} V until C/100",
    ) ]
# step index for RPT
step_RPT_CD = 1;  step_RPT_RE =2;   step_RPT_CC = 3;  
Update_Cycles = 3; RPT_Cycles = 1
Experiment_Long   = pybamm.Experiment( exp_AGE_text * Update_Cycles  )  
Experiment_RPT    = pybamm.Experiment( exp_RPT_text * RPT_Cycles     ) 
Experiment_Breakin= pybamm.Experiment( exp_RPT_text * RPT_Cycles     )


In [47]:
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 20,  # negative particle
    "r_p": 20,  # positive particle
}

In [48]:
sim1 = pybamm.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pybamm.CasadiSolver(),
    var_pts=var_pts,
)
solution1 = sim1.solve(
    save_at_cycles=1
)

stop = timeit.default_timer()
print('running time: ' + str(stop - start) +'s')

running time: 30.200252099999943s


In [49]:
dict_short = {}; 
list_short = []
# update 220808 to satisfy random model option:
for var, equation in model.initial_conditions.items():
    #print(var._name)
    list_short.append(var._name)
# delete Porosity times concentration and Electrolyte potential then add back
list_short.remove("Porosity times concentration");
list_short.remove("Electrolyte potential");
list_short.extend(
    ("Negative electrode porosity times concentration",
    "Separator porosity times concentration",
    "Positive electrode porosity times concentration",   

    "Negative electrolyte potential",
    "Separator electrolyte potential",
    "Positive electrolyte potential",))
for list_short_i in list_short:
    dict_short.update( { list_short_i : solution1.last_state[list_short_i].data  }  )

model_2 = model.set_initial_conditions_from(dict_short, inplace=False)
sim2 = pybamm.Simulation(
    model_2, 
    experiment=Experiment_Long,
    parameter_values=param,
    solver=pybamm.CasadiSolver(),
    var_pts=var_pts,
)
solution2 = sim2.solve(
    calc_esoh=False,
    save_at_cycles=1 
)

2023-02-20 23:40:16.012 - [ERROR] callbacks.on_experiment_error(226): Simulation error: Events ['Minimum voltage'] are non-positive at initial conditions


SolverError: Events ['Minimum voltage'] are non-positive at initial conditions

In [ ]:
solution2.cycles[0]['Current [A]'].entries

array([-1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
       -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
       -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
       -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
       -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
       -1.5, -1.5, -1.5, -1.5])

In [ ]:
solution2.cycles[0].steps[0]['Current [A]'].entries

TypeError: 'EmptySolution' object is not subscriptable